In [ ]:
#!/usr/bin/env python3
"""experiment_viterbi.ipynb
James Gardner 2019

applies viterbi analysis pipeline to experiment video
takes signal as time series of centre's intensity
"""

import cv2
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from tqdm import tqdm_notebook as tqdm

In [ ]:
def fourier_spectrum(signal, fps, return_spectrum=False,
                     produce_plot=False, out_plot_name='tmp.pdf', out_plot_title=''):
    """plots fourier spectrum of time series"""
    # implementation from tracker_time_series
    # after_first loses a frame
    
    signal_frames = len(signal)
    total_time = (signal_frames-2)/fps
    t = np.linspace(0,total_time,signal_frames)
    dt = t[1] - t[0]

    yf = np.fft.fft(signal)
    # normalised-absolute value of FT'd signal
    nrm_abs_yf = 2/signal_frames*np.abs(yf)
    freq_scale = np.fft.fftfreq(len(yf),dt)
    freq_scale_positive = freq_scale[:signal_frames//2]    
    # np.fft.fftfreq outputs 0 to +inf then -inf to 0, so :N//2 gets +ve side; wild!
    freq_prob = nrm_abs_yf[:signal_frames//2]
    
    if produce_plot:
        fig, (ax0,ax1) = plt.subplots(2,figsize=(14,14))
        ax0.plot(t,signal)
        ax0.set(title='signal: {}'.format(out_plot_title),ylabel='signal strength',xlabel='time, t')
        # filter out the average value gives magnitude of zero freq term
        ax1.plot(freq_scale_positive[2:],freq_prob[2:])
        ax1.set(title='discrete FFT',ylabel='freq strength in signal',xlabel='frequency, f')
        # REMEMBER TO CHANGE OUT FILENAME
        plt.savefig(out_plot_name,bbox_inches='tight')
        plt.clf()
    
    if return_spectrum:
        # cut after the frequency = 0 average value signal 
        return freq_prob[2:], freq_scale_positive[2:]        

In [ ]:
def series_at_point(filename, point=None, return_series=False,
                    produce_plot=False, out_plot_name='tmp.pdf'):
    """green channel intensity at point in the interference pattern
    also plots time series and spectrum thereof
    aka: run_dog_run, won't produce/return anything unless you tell it to
    """
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    pbar = tqdm(total=total_frames-1)

    point_intensity = []
    ret, frame = cap.read()
    # default to centre of frame
    if point is None:
        point = tuple([int(i/2) for i in frame.shape[:2]])

    while ret:
        # centre time series
        # use green channel since it has the greatest weight anyway
        # https://en.wikipedia.org/wiki/Relative_luminance
        point_intensity.append(frame[point][1])

        ret, frame = cap.read()
        pbar.update(1)

    pbar.close()
    cap.release()

    if produce_plot:
        fourier_spectrum(point_intensity, fps, produce_plot=True,
                         out_plot_name=out_plot_name, out_plot_title=filename)
    if return_series:
        return point_intensity, fps

In [ ]:
class PointViterbi(object):
    """finds viterbi path in frequency spectrum measured at centre point
    # apply viterbi to binned long capture
    # capture long video, slowly changing the frequency throughout (up then back down)
    # bin into 30 second slices, or 40 bins?
    # fourier transform
    # viterbi to recover meandering frequency"""
    def __init__(self,filename,long_timesteps,scanning_range):
        # note: some low frequency noise at high driving frequencies
        # might be due to poor measure at the centre
        # would be nice to take series just at the inside of the inner ring
        long_signal, fps = series_at_point(filename,return_series=True)
        total_frames = len(long_signal)
        duration = total_frames/fps
        # q,r = divmod(a,b) s.t. a = q*b+r
        bin_frames, bin_remainder = divmod(total_frames,long_timesteps)
        # assert total_frames == bin_frames*long_timesteps+bin_remainder
        # bin_duration = bin_frames/fps
        if bin_remainder == 0:
            bin_last = total_frames
        else:
            # tosses remainder
            #bin_last = total_frames - bin_frames
            bin_last = total_frames - bin_remainder - 1

        # always has long_timesteps number of chunks plus remainder which is tossed
        bin_signals = [long_signal[i: i+bin_frames] for i in range(0, bin_last, bin_frames)]

        # form up the grid
        # positive side of the fourier spectrum will have half bin_frames
        # minus 2 for cutting out the average value, frequency = 0, signal
        grid_frames = bin_frames//2-2
        grid = np.zeros((grid_frames,long_timesteps))

        for i,signal in enumerate(bin_signals):
            col, freq_scale_cut = fourier_spectrum(signal, fps, return_spectrum=True)
            grid[:,i] = col

        # normalised grid, trying to maximise product of values
        ngrid  = grid/np.max(grid)
        # logarithm avoids underflow, maximise sum of log(value)'s
        lngrid = np.log(ngrid)

        score_grid  = np.copy(lngrid)
        pathfinder_flag = len(lngrid[:,0]) #=500
        # pathfinder stores the survivor paths, to allow back-tracking through
        pathfinder = np.full(np.shape(lngrid), pathfinder_flag)
        # pathfinder flag+1 for reaching the first, 0-index column        
        pathfinder[:,0] = pathfinder_flag+1       

        # the viterbi algorithm, through time finding the best path to each node
        # see: https://www.youtube.com/watch?v=6JVqutwtzmo
        for j in range(1,long_timesteps): #range(100)
            for i in range(len(score_grid[:,j])): #range(500)
                # index values for where to look relative to i in previous column
                k_a = max(0, i-scanning_range) 
                k_b = min(len(score_grid[:,j-1])-1,
                          i+scanning_range)
                #print(k_a,k_b)
                window = score_grid[:,j-1][k_a:k_b+1]
                # find the best thing nearby in the previous column ...
                window_score = np.max(window)
                window_ref   = k_a+np.argmax(window)
                # ... and take note of it, summing the log(value)'s
                score_grid[i][j] += window_score
                pathfinder[i][j] = window_ref 

        # look at the very last column, and find the best ending for the path
        best_score  = np.max(score_grid[:,-1])
        best_end = np.argmax(score_grid[:,-1])
        # now need to retrace the steps through the grid
        best_path_back = np.full(long_timesteps,pathfinder_flag+2)
        best_path_back[-1] = best_end

        # path_grid is the binary image of the viterbi path taken
        path_grid = np.zeros(np.shape(ngrid))
        tmp_path = pathfinder[best_end][-1]

        for j in reversed(range(0,long_timesteps-1)):
            path_grid[tmp_path][j] = 1
            # take pathfinder value in current step and follow it backwards
            best_path_back[j] = tmp_path    
            tmp_path = pathfinder[tmp_path][j]

        # make sure we got all the way home
        assert tmp_path == pathfinder_flag+1
        
        self.ngrid = ngrid
        self.path_grid = path_grid        
        self.plot_bundle = filename, long_timesteps, duration, grid_frames, freq_scale_cut 

    def plot(self, filetag='tmp.pdf'):
        """saves plots of meandering frequency, the signal grid, and the recovered viterbi path"""
        filename, long_timesteps, duration, grid_frames, freq_scale_cut = self.plot_bundle
        
        plt.figure(figsize=(7,14))
        plt.imshow(self.ngrid, cmap='viridis')
        plt.gca().xaxis.tick_top()
        plt.gca().xaxis.set_label_position('top')

        tick_skip = int(duration//100)
        xtick_labels_0 = (np.linspace(0,1,long_timesteps)*duration).astype(int)
        xtick_labels_1 = []
        for i in range(0,long_timesteps,tick_skip):
            xtick_labels_1.append(xtick_labels_0[i])
            for _ in range(tick_skip):
                xtick_labels_1.append('')
        ytick_labels_0 = ['{:.2f}'.format(i) for i in freq_scale_cut]
        ytick_labels_1 = []
        for i in range(0,grid_frames,tick_skip):
            ytick_labels_1.append(ytick_labels_0[i])
            for _ in range(tick_skip):
                ytick_labels_1.append('')    
        plt.xticks(np.arange(long_timesteps),xtick_labels_1, rotation=90)
        plt.yticks(np.arange(grid_frames),ytick_labels_1)
        
        cbar_fraction = 0.025
        cbar = plt.colorbar(fraction=cbar_fraction) 
        cbar.set_label('normalised frequency distribution')
        plt.title('{}\n fourier spectrum of signal binned over time\n'.format(filename))
        plt.ylabel('signal frequency, f / Hz')
        plt.xlabel('long time duration, t / s')
        plt.savefig('expt_ngrid_{}.pdf'.format(filetag),bbox_inches='tight')
        plt.clf();
        
        plt.figure(figsize=(7,14))
        plt.imshow(self.path_grid, cmap='viridis')
        plt.gca().xaxis.tick_top()
        plt.gca().xaxis.set_label_position('top')  
        plt.xticks(np.arange(long_timesteps),xtick_labels_1, rotation=90)
        plt.yticks(np.arange(grid_frames),ytick_labels_1)
        plt.title('{}\n viterbi path through signal grid'.format(filename))
        plt.ylabel('signal frequency, f / Hz')
        plt.xlabel('long time duration, t / s')
        plt.savefig('expt_viterbi_path_{}.pdf'.format(filetag),bbox_inches='tight')
        plt.clf();

In [ ]:
# first milestone    
PointViterbi('expt_5_fast.mp4',long_timesteps=50,scanning_range=3).plot('5_fast')
PointViterbi('expt_5_slow.mp4',long_timesteps=100,scanning_range=3).plot('5_slow')
PointViterbi('expt_5_high.mp4',long_timesteps=100,scanning_range=3).plot('5_high')
PointViterbi('expt_5_long.mp4',long_timesteps=200,scanning_range=3).plot('5_long')